# LLM Issue Domain Classification

## Installs

In [3]:
%pip install pandas emoji openai tiktoken scikit-learn

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/b1/10/c04b4ff592a5f46b28ebf4c2353f735c02ae7f0ce1b165d00748ced6467e/tiktoken-0.7.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tiktoken-0.7.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/799.0 kB ? eta -:--:--
    --------------------------------------- 10.2/799.0 kB ? eta -:--:--
    --------------------------------------- 10.2/799.0 kB ? eta -:--:--
    --------------------------------------- 10.2/799.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/799.0 kB 186.2 kB/s eta 0:00:05
   -- ------------------------------------ 61.4/799.0 kB 272.3 kB/s eta 0:00:03
   ----- -------------------------------- 112.6/799.0 kB 435.7 kB/s eta 0:00:02
   -------- ----------------------------- 184.3/799.0 kB 617.3 kB/s eta 0:00:01
   -------------- ----------------------- 297.0/799.0 kB 833.5 kB/s eta 0:00:01
   -------------------- -----


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Library Imports

In [1]:
# Importing libraries
import pandas as pd
import emoji
import re
import string
import json


## Data Import

In [2]:
issue_data = pd.read_csv("core_engine_out.csv")


issue_data.head()

,PR #,Pull Request,issue text,issue description,created_at,closed_at,userlogin,author_name,most_recent_commit,filename,...,Utility-Performance Tools,Utility-Diagnostic Utilities,Utility-Backup Tools,Test,Test-Unit Testing,Test-Integration Testing,Test-Performance Testing,Test-Security Testing,Test-Usability Testing,Test-Regression Testing
0,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
1,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
2,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
3,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/FileSortTab.java,...,0,0,0,0,0,0,0,0,0,0
4,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/FileSortTab.java,...,0,0,0,0,0,0,0,0,0,0


#### Create DF With only Issue Title and Body

For the first approach I feel we should use only issue title and body as these are the two features that will be present in an issue at every point throughout the issue's life. PR related features as well as issue comments likely won't exist when an issue is first created, so excluding those will ensure the robustness of our model.

In [3]:
df = issue_data

df.head()

,PR #,Pull Request,issue text,issue description,created_at,closed_at,userlogin,author_name,most_recent_commit,filename,...,Utility-Performance Tools,Utility-Diagnostic Utilities,Utility-Backup Tools,Test,Test-Unit Testing,Test-Integration Testing,Test-Performance Testing,Test-Security Testing,Test-Usability Testing,Test-Regression Testing
0,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
1,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
2,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
3,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/FileSortTab.java,...,0,0,0,0,0,0,0,0,0,0
4,1,True,New Sorting/Export preferences,"This will add a new ""File Sorting"" Tab to the ...",2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/FileSortTab.java,...,0,0,0,0,0,0,0,0,0,0


## Preprocessing

Preprocessing steps involve cleaning the issue title and body text. On first approach I am using the approach found in the folowing project, https://github.com/G4BE-334/NLBSE-issue-report-classification/blob/main/issueclassificationgpt.ipynb 
This is doing similar work in labeling github issues and provides a way to remove noise commonly found in github text (html tags, links, etc.)

In [4]:
cleaned_count = 0
original_count = 0

# Text cleaning function
def clean_text(text):
    global cleaned_count, original_count

    if not isinstance(text, str):
        original_count += 1
        return text

    # Remove double quotation marks
    text = text.replace('"', '')

    # Remove text starting with "DevTools" and ending with "(automated)"
    text = re.sub(r'DevTools.*?\(automated\)', '', text)

    # Lowercasing should be one of the first steps to ensure uniformity
    text = text.lower()

    # Remove emojis
    text = emoji.demojize(text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters and punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)

    # Remove '#' characters
    text = text.replace("#", "")

    # Remove consecutive whitespaces and replace with a single space
    text = re.sub(r'\s+', ' ', text)

    # Split the text into words
    words = text.split()

    # Remove words that are over 20 characters
    words = [word for word in words if len(word) <= 20]

    # Join the remaining words back into cleaned text
    cleaned_text = ' '.join(words)

    cleaned_count += 1
    return cleaned_text

df['issue text'] = df['issue text'].apply(clean_text)
df['issue description'] = df['issue description'].apply(clean_text)
df.head()

,PR #,Pull Request,issue text,issue description,created_at,closed_at,userlogin,author_name,most_recent_commit,filename,...,Utility-Performance Tools,Utility-Diagnostic Utilities,Utility-Backup Tools,Test,Test-Unit Testing,Test-Integration Testing,Test-Performance Testing,Test-Security Testing,Test-Usability Testing,Test-Regression Testing
0,1,True,new sortingexport preferences,this will add a new file sorting tab to the pr...,2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
1,1,True,new sortingexport preferences,this will add a new file sorting tab to the pr...,2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
2,1,True,new sortingexport preferences,this will add a new file sorting tab to the pr...,2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/imports/HTMLConverter.java,...,0,0,0,0,0,0,0,0,0,0
3,1,True,new sortingexport preferences,this will add a new file sorting tab to the pr...,2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/FileSortTab.java,...,0,0,0,0,0,0,0,0,0,0
4,1,True,new sortingexport preferences,this will add a new file sorting tab to the pr...,2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/FileSortTab.java,...,0,0,0,0,0,0,0,0,0,0


## Data Split

We will split the data 70/30 to prepare for training and testing steps

In [5]:
# Randomly sample 70% of the rows for the first DataFrame
training_df = df.sample(frac=0.7, random_state=1)

# The remaining 30% of the rows for the second DataFrame
testing_df = df.drop(training_df.index)

training_df.head()


,PR #,Pull Request,issue text,issue description,created_at,closed_at,userlogin,author_name,most_recent_commit,filename,...,Utility-Performance Tools,Utility-Diagnostic Utilities,Utility-Backup Tools,Test,Test-Unit Testing,Test-Integration Testing,Test-Performance Testing,Test-Security Testing,Test-Usability Testing,Test-Regression Testing
457,8,True,pdffile metadata privacy filtering all metadata,this pullrequest pertains to the addition of m...,2014-04-23T12:58:24Z,2014-04-23T21:23:03Z,adaerr,Adrian Daerr,2360cb74d2e8d88e1bc705db36e885e44607642f,src/main/java/net/sf/jabref/util/XMPUtil.java,...,0,0,0,0,0,0,0,0,0,0
492,9,True,support findfulltext with acs dois,adds a fulltextfinder implementation to transf...,2014-05-06T06:36:03Z,2014-05-20T12:53:43Z,ansell,Peter Ansell,57cbe70e1be14f7b8f8754e05f6b722ed72ce887,src/main/java/net/sf/jabref/external/ACSPdfDow...,...,0,0,0,0,0,0,0,0,0,0
550,10,True,try to fix some obvious bugs about groups,hi all i just upgraded to 210 and i found some...,2014-05-08T03:14:52Z,2014-05-20T12:55:20Z,braindevices,Ling Wang,b6a89a81a983332d35420534a766d8dfabb6b81e,src/main/java/net/sf/jabref/gui/GroupAddRemove...,...,0,0,0,0,0,0,0,0,0,0
516,10,True,try to fix some obvious bugs about groups,hi all i just upgraded to 210 and i found some...,2014-05-08T03:14:52Z,2014-05-20T12:55:20Z,braindevices,Ling Wang,b6a89a81a983332d35420534a766d8dfabb6b81e,src/main/java/net/sf/jabref/JabRefPreferences....,...,0,0,0,0,0,0,0,0,0,0
162,1,True,new sortingexport preferences,this will add a new file sorting tab to the pr...,2014-03-12T09:26:29Z,2014-03-12T11:38:01Z,olenz,Olaf Lenz,6db780e96882aca9fa3170afd13a7965dfb61a69,src/java/net/sf/jabref/export/FileActions.java,...,0,0,0,0,0,0,0,0,0,0


## Data Transformation

Generate messages based on our dataset that will be used to fine-tune the gpt model, ensuring that the messages are clear in what we want the model to do.

Load in domains and reformat the json for clarity, for the first test I feel we should will the descriptions as I feel they provide more context for the model. This will also increase the price due to there being more words, be we can see how significant the difference is later.

In [6]:
with open("Domains.json", 'r') as file:
    dictionary = json.load(file)


In [7]:
formatted_domains = []
gpt_output = {}
# reformat domains to increase clarity for gpt model and create dictionary with only domains/subdomains (to serve as expected gpt output)
for key, value in dictionary.items():
    gpt_output[key] = 0
    
    temp = {}
    temp["Domain"] = key
    temp["SubDomains"] = {}
    for i in range(len(value)):
        subdomain, description = list(value[i].items())[0]
        temp["SubDomains"][subdomain] = description
        gpt_output[subdomain] = 0

    formatted_domains.append(temp)


# convert to string to pass to gpt model
domains_string = str(formatted_domains)
print(domains_string)

[{'Domain': 'Application', 'SubDomains': {'Application-Integration': 'Pertains to combining the application with other systems or platforms for enhanced functionality.', 'Application-Plugin Management': "Involves handling, updating, and configuring add-ons that extend the application's capabilities.", 'Application-User Customization': "Allows users to modify the application's settings or features according to their preferences.", 'Application-App Configuration': 'Focuses on setting up initial application settings and adjusting features to meet specific needs.', 'Application-Version Control': 'Deals with managing changes to application documents, programs, and other information stored as part of the development process.', 'Application-Compatibility Checks': 'Ensures the application functions correctly across different devices, operating systems, or with other applications.'}}, {'Domain': 'Application Performance Manager', 'SubDomains': {'Application Performance Manager-Performance Monit

In [8]:
print(df.columns)

Index(['PR #', 'Pull Request', 'issue text', 'issue description', 'created_at',
       'closed_at', 'userlogin', 'author_name', 'most_recent_commit',
       'filename',
       ...
       'Utility-Performance Tools', 'Utility-Diagnostic Utilities',
       'Utility-Backup Tools', 'Test', 'Test-Unit Testing',
       'Test-Integration Testing', 'Test-Performance Testing',
       'Test-Security Testing', 'Test-Usability Testing',
       'Test-Regression Testing'],
      dtype='object', length=232)


Messages will then be stored in a json as this is a suitable format for training the gpt model.

In [10]:

# Open the file in write mode
with open('data.jsonl', 'w', encoding='utf-8') as f:
    assistant_message = gpt_output
    # Iterate over the rows in the DataFrame
    for index, row in training_df.iterrows():
        # Create the user message by formatting the prompt with the title and body
        user_message = (
        f"Classify a GitHub issue by indicating whether each domain and subdomain is relevant to the issue based on its title: [{row['issue text']}] "
        f"and body: [{row['issue description']}]. Ensure that every domain/subdomain is accounted for, and its relevance is indicated with a 1 (relevant) or a 0 (not relevant)."
        )

        # logic to update assistant message with values in df
        for column in df.columns:
            if column in gpt_output:
                assistant_message[column] = row[column]

        if(row['PR #'] == 13):
            print(str(row['PR #']) + ':'+ str(assistant_message))
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "Refer to these domains and subdomains when classifying" + domains_string},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": str(assistant_message)}
                ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

13:{'Application': 0, 'Application-Integration': 0, 'Application-Plugin Management': 0, 'Application-User Customization': 0, 'Application-App Configuration': 0, 'Application-Version Control': 0, 'Application-Compatibility Checks': 0, 'Application Performance Manager': 0, 'Application Performance Manager-Performance Monitoring': 0, 'Application Performance Manager-Resource Allocation': 0, 'Application Performance Manager-Error Detection': 0, 'Application Performance Manager-Load Balancing': 0, 'Application Performance Manager-Traffic Management': 0, 'Application Performance Manager-Diagnostic Tools': 0, 'Big Data': 0, 'Big Data-Data Processing': 0, 'Big Data-Data Storage': 0, 'Big Data-Data Analysis': 0, 'Big Data-Real-Time Processing': 0, 'Big Data-Batch Processing': 0, 'Big Data-Data Visualization': 0, 'Cloud': 0, 'Cloud-Resource Management': 0, 'Cloud-Virtualization': 0, 'Cloud-Scalability Solutions': 0, 'Cloud-Cloud Security': 0, 'Cloud-Data Migration': 0, 'Cloud-Service Configurati